In [1]:
from lxml import etree
from urdfpy import URDF,Link,Joint
import numpy as np
from scipy.spatial.transform import Rotation
import yaml
import os
import copy
import urdfpy

In [16]:
ur51=robot = URDF.load('/home/vignesh/curobo/src/curobo/content/assets/robot/ur_description/ur5e.urdf')

In [17]:
for link in ur51.links:
    link.name=link.name+"_1"

In [18]:
for link in ur51.links:
    print(link.name)

base_link_1
base_link_inertia_1
shoulder_link_1
upper_arm_link_1
forearm_link_1
wrist_1_link_1
wrist_2_link_1
wrist_3_link_1
base_1
flange_1
tool0_1


In [19]:
for joint in ur51.joints:
    joint.name=joint.name+"_1"
    joint.parent=joint.parent+"_1"
    joint.child=joint.child+"_1"
for joint in ur51.joints:
    print('{} connects {} to {}'.format(joint.name, joint.parent, joint.child))

base_link-base_link_inertia_1 connects base_link_1 to base_link_inertia_1
shoulder_pan_joint_1 connects base_link_inertia_1 to shoulder_link_1
shoulder_lift_joint_1 connects shoulder_link_1 to upper_arm_link_1
elbow_joint_1 connects upper_arm_link_1 to forearm_link_1
wrist_1_joint_1 connects forearm_link_1 to wrist_1_link_1
wrist_2_joint_1 connects wrist_1_link_1 to wrist_2_link_1
wrist_3_joint_1 connects wrist_2_link_1 to wrist_3_link_1
base_link-base_fixed_joint_1 connects base_link_1 to base_1
wrist_3-flange_1 connects wrist_3_link_1 to flange_1
flange-tool0_1 connects flange_1 to tool0_1


In [21]:
ur51.base_link.name

'base_link_1'

In [27]:
ur52=URDF.load('/home/vignesh/curobo/src/curobo/content/assets/robot/ur_description/ur5e.urdf')

In [28]:
ur52=clean_robot(ur52,"_2")

In [29]:
for joint in ur52.joints:
    print('{} connects {} to {}'.format(joint.name, joint.parent, joint.child))

base_link-base_link_inertia_2 connects base_link_2 to base_link_inertia_2
shoulder_pan_joint_2 connects base_link_inertia_2 to shoulder_link_2
shoulder_lift_joint_2 connects shoulder_link_2 to upper_arm_link_2
elbow_joint_2 connects upper_arm_link_2 to forearm_link_2
wrist_1_joint_2 connects forearm_link_2 to wrist_1_link_2
wrist_2_joint_2 connects wrist_1_link_2 to wrist_2_link_2
wrist_3_joint_2 connects wrist_2_link_2 to wrist_3_link_2
base_link-base_fixed_joint_2 connects base_link_2 to base_2
wrist_3-flange_2 connects wrist_3_link_2 to flange_2
flange-tool0_2 connects flange_2 to tool0_2


In [6]:
def clean_robot(urdf_object,append_string="_1"):
    for link in urdf_object.links:
        link.name=link.name+append_string
    for joint in urdf_object.joints:
        joint.name=joint.name+append_string
        joint.parent=joint.parent+append_string
        joint.child=joint.child+append_string
    return urdf_object
def pose_list_to_pose_matrix(pose):
    pose_matrix=np.eye(4)
    rot=Rotation.from_quat(np.array(pose)[[4,5,6,3]])
    
    pose_matrix[0:3,3]=pose[0:3]
    pose_matrix[0:3,0:3]=rot.as_matrix()
    return pose_matrix
def add_robots_urdf(urdf_list,pose_list,name):
    assert len(urdf_list)==len(pose_list),'inputs should be of same length'
    base_link=Link('base_fixture_link',None,visuals=[urdfpy.Visual(urdfpy.Geometry(sphere=urdfpy.Sphere(0.1)))],collisions=None)
    new_links=[base_link]
    new_joints=[]
    for i in range(len(urdf_list)):
        robot_pose=pose_list[i]
        urdf_list[i]=clean_robot(urdf_list[i],"_"+str(i+1))
        new_links+=urdf_list[i].links
        new_joints+=[Joint(name=base_link.name+"_j_"+urdf_list[i].base_link.name,joint_type='fixed',parent=base_link.name,child=urdf_list[i].base_link.name,origin=list(pose_list_to_pose_matrix(robot_pose)))]
        new_joints+=urdf_list[i].joints
    #for links in new_links:
        #print(link.name)
    #kinematics={}
    return URDF(name,links=new_links,joints=new_joints)
#def add_robots(config_list,urdf_path):
    
    

In [3]:
robot_list=[URDF.load('/home/vignesh/curobo/src/curobo/content/assets/robot/ur_description/ur5e_new.urdf'),URDF.load('/home/vignesh/curobo/src/curobo/content/assets/robot/ur_description/ur5e_new.urdf')]
pose_list=[[-0.75,-0.5,0,1,0,0,0],[0.75,0.5,0,0,0,0,1]]
#clean_robot(robot_list[0],'_'+str(2)).joints

In [7]:
final=add_robots_urdf(robot_list,pose_list,name='dual')

In [8]:
final.show()

In [8]:
with open('/home/vignesh/curobo/src/curobo/content/configs/robot/ur5e.yml', 'r') as f:
    robot = yaml.load(f, Loader=yaml.SafeLoader)
robot.keys()

dict_keys(['robot_cfg'])

In [9]:
robot=robot['robot_cfg']['kinematics']

In [11]:
print(robot['link_names'])

None


In [7]:
new_spheres={key+"_1":value for key,value in robot['collision_spheres'].items()}
new_spheres[['shoulder_link_1','upper_arm_link_1']]

TypeError: unhashable type: 'list'

In [2]:
import torch
import curobo
from curobo.types.math import Pose
from curobo.types.base import TensorDeviceType
from curobo.types.robot import JointState
from curobo.wrap.reacher.motion_gen import MotionGen, MotionGenConfig, MotionGenPlanConfig
from curobo.geom.sdf.world import CollisionCheckerType
from curobo.geom.types import WorldConfig, Cuboid, Mesh, Capsule, Cylinder, Sphere
from curobo.util_file import get_assets_path, join_path
%reload_ext autoreload

In [3]:
motion_gen_config_1 = MotionGenConfig.load_from_robot_config(
    "ur5e_new.yml",
    {'cuboid':{'table_collision': {'dims': [3.2, 0.8, 0.1],
   'pose': np.array([-0.0,  0.8, -0.01,  1.  , -0.  , -0.  , -0.  ])}}},
    interpolation_dt=0.05,
    collision_checker_type=CollisionCheckerType.PRIMITIVE,collision_activation_distance=0.025,trajopt_dt=0.25,
)
motion_gen1 = MotionGen(motion_gen_config_1)
motion_gen1.warmup(parallel_finetune=True)

In [53]:
def combine_yaml_kinematics(config_list,combined_urdf_path,config_path):
    kinematic_list=[]
    for config in config_list:
        kinematic_list.append(config['robot_cfg']['kinematics'])
    new_kinematics={'urdf_path':combined_urdf_path,'asset_root_path':'/robot','base_link':'base_fixture_link'}
    #new_lock_joints={}
    collison_sphere={} #
    ee_link=kinematic_list[0]['ee_link']+"_1"
    links_names=[]#
    lock_joints={}#
    extra_links={}#
    collison_sphere_buffer=0#
    extra_collison_spheres={}#
    self_collison_ignore={}#
    self_collison_buffer={}#
    mesh_link_names=[]#
    collision_link_names=[]#
    use_global_cumul=False#
    cspace={'joint_names':[],'retract_config':[],'null_space_weight':[],'cspace_distance_weight':[],'max_jerk':float('inf'),'max_acceleration':float('inf')}
    for i in range(len(kinematic_list)):
        if isinstance(kinematic_list[i]['collision_spheres'],str):
            with open(os.path.join(config_path,'spheres'), 'r') as f:
                spheres_i = yaml.load(f, Loader=yaml.SafeLoader)['collision_spheres']
                
        else:
            spheres_i=kinematic_list[i]['collision_spheres']
        spheres_i={key+"_"+str(i+1):value for key,value in spheres_i.items()}
        collison_sphere.update(spheres_i)
        links_names.append(kinematic_list[i]['ee_link']+'_'+str(i+1))
        if kinematic_list[i]['lock_joints'] is not None:
            lock_joints.update({key+"_"+str(i+1):value for key,value in kinematic_list[i]['collision_spheres'].items()})
        if kinematic_list[i]['extra_links'] is not None:
            for k,v in kinematic_list[i]['extra_links'].items():
                v['parent_link_name']+="_"+str(i+1)
                v['link_name']+="_"+str(i+1)
                v['joint_name']+="_"+str(i+1)
                extra_links[k+"_"+str(i+1)]=v
        collison_sphere_buffer=max(collison_sphere_buffer,kinematic_list[i]['collision_sphere_buffer'])
        if kinematic_list[i]['extra_collision_spheres'] is not None:
            extra_collison_spheres.update({key+"_"+str(i+1):value for key,value in kinematic_list[i]['extra_collision_spheres'].items()})
        self_collison_ignore.update({key+"_"+str(i+1):[vi+"_"+str(i+1) for vi in value] for key,value in kinematic_list[i]['self_collision_ignore'].items()})
        self_collison_buffer.update({key+"_"+str(i+1):value for key,value in kinematic_list[i]['self_collision_buffer'].items()})
        mesh_link_names+=[name+"_"+str(i+1) for name in kinematic_list[i]['mesh_link_names']]
        collision_link_names+=[name+"_"+str(i+1) for name in kinematic_list[i]['collision_link_names']]
        use_global_cumul=(use_global_cumul or kinematic_list[i]['use_global_cumul'])
        cspace['joint_names']+=[name+"_"+str(i+1) for name in kinematic_list[i]['cspace']['joint_names']]
        cspace['retract_config']+=kinematic_list[i]['cspace']['retract_config']
        cspace['null_space_weight']+=kinematic_list[i]['cspace']['null_space_weight']
        cspace['cspace_distance_weight']+=  kinematic_list[i]['cspace']['cspace_distance_weight']
        cspace['max_jerk']=min(cspace['max_jerk'],kinematic_list[i]['cspace']['max_jerk'])
        cspace['max_acceleration']=min(cspace['max_acceleration'],kinematic_list[i]['cspace']['max_acceleration'])
    new_kinematics['collision_spheres']=collison_sphere
    new_kinematics['ee_link']=ee_link
    new_kinematics['link_names']=links_names
    new_kinematics['lock_joints']=lock_joints
    new_kinematics['extra_links']=extra_links
    new_kinematics['collision_sphere_buffer']=collison_sphere_buffer
    new_kinematics['extra_collision_spheres']=extra_collison_spheres
    new_kinematics['self_collision_ignore']=self_collison_ignore
    new_kinematics['self_collision_buffer']=self_collison_buffer
    new_kinematics['mesh_link_names']=mesh_link_names
    new_kinematics['collision_link_names']=collision_link_names
    new_kinematics['use_global_cumul']=use_global_cumul
    new_kinematics['cspace']=cspace
    return {'robot_cfg':{'kinematics':new_kinematics}}
        
        
        
        
        
            
            
        
        
    

In [54]:
with open('/home/vignesh/curobo/src/curobo/content/configs/robot/ur5e_new.yml', 'r') as f:
    ur5e_config = yaml.load(f, Loader=yaml.SafeLoader)
ur5e_config.keys()

dict_keys(['robot_cfg'])

In [55]:
robot_list=[URDF.load('/home/vignesh/curobo/src/curobo/content/assets/robot/ur_description/ur5e_new.urdf'),URDF.load('/home/vignesh/curobo/src/curobo/content/assets/robot/ur_description/ur5e_new.urdf')]
pose_list=[[-0.75,0,0,1,0,0,0],[0.75,0,0,0.7071068,0,0, 0.7071068]]
final=add_robots_urdf(robot_list,pose_list,name='dual_ur5e_new')
final.save('/home/vignesh/curobo/src/curobo/content/assets/robot/ur_description/dual_ur5e_new.urdf')

In [56]:
ur51_config=copy.deepcopy(ur5e_config)
ur52_config=copy.deepcopy(ur5e_config)
new_yaml=combine_yaml_kinematics([ur51_config,ur52_config],'robot/ur_description/dual_ur5e_new.urdf','~/curobo/src/curobo/content/configs/robot')


In [48]:
with open('/home/vignesh/curobo/src/curobo/content/configs/robot/dual_ur5e_new.yml', 'w') as outfile:
    yaml.dump(new_yaml, outfile, default_flow_style=False)

In [47]:
os.path.exists('/home/vignesh/curobo/src/curobo/content/configs')

True